# Markup Data Analysis

This notebook explores the markup data and performs analysis on the quote and order data.

In [1]:
# Initial imports
import sys
import os
import pandas as pd
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Add project src directory to path
# sys.path.append(os.path.abspath(os.path.join('..', 'src')))
os.chdir('..')

# Import configuration class
from gomat_markup_opt.config import ConversionModelTrainingSettings

# Instantiate configuration
config = ConversionModelTrainingSettings()


In [2]:
# Import necessary functions for data loading and processing
from gomat_markup_opt.data.preparation import load_and_prepare_quote_order_data
from gomat_markup_opt.preprocessing.feature_engineering import engineer_features
from gomat_markup_opt.training.train_conversion_model import _split_dataset, _balance_data
from gomat_markup_opt.training.data_augmentation import generate_synthetic_data
from gomat_markup_opt.preprocessing.cleanup import cleanup_and_filter_quote_and_order_data

from gomat_markup_opt.logging_config import setup_logging

# Define target and markup columns
TARGET_COLUMN = "target"
MARKUP_COLUMN = "markup_rate"

setup_logging()

In [3]:
print("****************************")
# Load and prepare quote and order data
prepared_quote_order_data = load_and_prepare_quote_order_data()
print(f"Rows after loading and preparing quote/order data: {len(prepared_quote_order_data)}")

****************************
2025-08-05 16:31:55 - gomat_markup_opt.data.preparation - INFO - Loading raw quote/order data...


C:\Users\sayed\Documents\Projects\markupml\src\gomat_markup_opt\data\loading.py:76: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, on_bad_lines="error")


2025-08-05 16:31:56 - gomat_markup_opt.data.preparation - INFO - Raw quote/order data shape: (323351, 40)
2025-08-05 16:32:02 - gomat_markup_opt.data.preparation - INFO - Prepared quote/order data shape after rename/select: (323351, 42)
2025-08-05 16:32:02 - gomat_markup_opt.data.preparation - INFO - Loading raw quote lost data...
2025-08-05 16:32:02 - gomat_markup_opt.data.preparation - INFO - Raw quote lost data shape: (598, 7)
2025-08-05 16:32:02 - gomat_markup_opt.data.preparation - INFO - Prepared quote lost data shape after rename/select: (598, 7)
2025-08-05 16:32:02 - gomat_markup_opt.data.preparation - INFO - Merging quote lost data with quote/order data...
2025-08-05 16:32:02 - gomat_markup_opt.data.preparation - INFO - Prepared quote/order data shape after merge: (323351, 43)
Rows after loading and preparing quote/order data: 323351


In [4]:
print("****************************")
# Engineer features
engineered_data = engineer_features(prepared_quote_order_data, target_column=TARGET_COLUMN)
print(f"Rows after feature engineering: {len(engineered_data)}")

****************************
2025-08-05 16:32:02 - gomat_markup_opt.preprocessing.feature_engineering - INFO - --- Engineering Features ---
2025-08-05 16:32:03 - gomat_markup_opt.preprocessing.feature_engineering - WARNING - Unmapped quote statuses encountered: {'Order Cancelled', 'Order Work In Progress', 'New Order'}
2025-08-05 16:32:03 - gomat_markup_opt.preprocessing.feature_engineering - WARNING - Removing 12204 rows with unmapped statuses out of 323351 total rows.
2025-08-05 16:32:03 - gomat_markup_opt.preprocessing.feature_engineering - INFO - Row count after removing unmapped statuses: 311147
2025-08-05 16:32:03 - gomat_markup_opt.preprocessing.feature_engineering - INFO - Columns in engineered data: ['item_id', 'type', 'request_purpose', 'qto', 'order_number', 'category_id', 'plant_category', 'product_id', 'latin_name', 'common_name', 'product_size_id', 'size', 'product_spec_id', 'specs', 'quantity', 'seller_price', 'buyer_price', 'seller_company_id', 'supplier_name', 'buyer_c

In [5]:
print("****************************")
cleaned_data = cleanup_and_filter_quote_and_order_data(engineered_data, cutoff_date=config.history_cutoff_date)
print(f"Rows after cleaning and filtering: {len(cleaned_data)}")

****************************
2025-08-05 16:32:03 - gomat_markup_opt.preprocessing.cleanup - INFO - --- Cleaning and Filtering Quote/Order Data ---
2025-08-05 16:32:03 - gomat_markup_opt.preprocessing.cleanup - INFO - Removed 2815 rows with non-positive seller_price (remaining: 308332)
2025-08-05 16:32:03 - gomat_markup_opt.preprocessing.cleanup - INFO - Removed 0 rows with invalid order_date (remaining: 308332)
2025-08-05 16:32:03 - gomat_markup_opt.preprocessing.cleanup - INFO - Removed 163687 rows with missing delivery_zip_code (remaining: 144645)
2025-08-05 16:32:03 - gomat_markup_opt.preprocessing.cleanup - INFO - Filtering to data after cutoff date: 2022-11-01
2025-08-05 16:32:03 - gomat_markup_opt.preprocessing.cleanup - INFO - Removed 31069 rows outside date range (remaining: 113576)
2025-08-05 16:32:03 - gomat_markup_opt.preprocessing.cleanup - INFO - Removed 45160 rows outside top regions (remaining: 68416)
2025-08-05 16:32:03 - gomat_markup_opt.preprocessing.cleanup - INFO - 

In [6]:
print("****************************")
# Split dataset into train and test
train_data, test_data = _split_dataset(cleaned_data, size_in_month=config.test_size)
print(f"Rows in train data after split: {len(train_data)}")
print(f"Rows in test data after split: {len(test_data)}")


****************************
Rows in train data after split: 57115
Rows in test data after split: 3969


In [7]:
print("****************************")
# Generate synthetic data
augmented_data = generate_synthetic_data(train_data, TARGET_COLUMN, MARKUP_COLUMN, config)
print(f"Rows after synthetic data generation: {len(augmented_data)}")


****************************
2025-08-05 16:32:03 - gomat_markup_opt.training.data_augmentation - INFO - Accepted high markups: 1971
2025-08-05 16:32:03 - gomat_markup_opt.training.data_augmentation - INFO - Rejected low markups: 6755
2025-08-05 16:32:03 - gomat_markup_opt.training.data_augmentation - INFO - Synthetic samples to generate: 0 (Accepted High -> Low Markup)
2025-08-05 16:32:03 - gomat_markup_opt.training.data_augmentation - INFO - Synthetic samples to generate: 0 (Rejected Low -> High Markup)
2025-08-05 16:32:03 - gomat_markup_opt.training.data_augmentation - INFO - No synthetic samples generated for Accepted High -> Low Markup (n_synthetic_1 == 0)
2025-08-05 16:32:03 - gomat_markup_opt.training.data_augmentation - INFO - No synthetic samples generated for Rejected Low -> High Markup (n_synthetic_2 == 0)
2025-08-05 16:32:03 - gomat_markup_opt.training.data_augmentation - INFO - Shape after augmentation: (57115, 49)
Rows after synthetic data generation: 57115


In [8]:
print("****************************")
# Split augmented data into train and calibration
train_data, calib_data = _split_dataset(augmented_data, size_in_month=6)
print(f"Rows in train data after augmentation split: {len(train_data)}")
print(f"Rows in calibration data after augmentation split: {len(calib_data)}")


****************************
Rows in train data after augmentation split: 35824
Rows in calibration data after augmentation split: 21291


In [9]:
print("****************************")
# Balance the train data
balanced_train_data = _balance_data(train_data, TARGET_COLUMN, config)
print(f"Rows in balanced train data: {len(balanced_train_data)}")

****************************
2025-08-05 16:32:03 - gomat_markup_opt.training.train_conversion_model - INFO - Target distribution before balancing: target
0    0.791536
1    0.208464
Name: proportion, dtype: float64
2025-08-05 16:32:03 - gomat_markup_opt.training.train_conversion_model - INFO - Undersampled refused quotes from 28356 to 14936
2025-08-05 16:32:03 - gomat_markup_opt.training.train_conversion_model - INFO - Target distribution after balancing: target
0    0.666667
1    0.333333
Name: proportion, dtype: float64
2025-08-05 16:32:03 - gomat_markup_opt.training.train_conversion_model - INFO - Shape after balancing: (22404, 49)
Rows in balanced train data: 22404


In [10]:
engineered_data["target"]

0         0
1         0
2         0
3         0
4         0
         ..
323277    1
323279    1
323282    1
323318    1
323324    1
Name: target, Length: 311147, dtype: int64

In [11]:
config.augment_high_markup_threshold

0.5

In [12]:
data = train_data
target_column = TARGET_COLUMN
markup_column = MARKUP_COLUMN

accepted_high_markups_mask = (data[markup_column] > config.augment_high_markup_threshold) & (
    data[target_column] == 1
)
rejected_low_markups_mask = (data[markup_column] < config.augment_low_markup_threshold) & (data[target_column] == 0)

accepted_high_markups = data[accepted_high_markups_mask]
rejected_low_markups = data[rejected_low_markups_mask]

print(f"Accepted high markups: {len(accepted_high_markups)}")
print(f"Rejected low markups: {len(rejected_low_markups)}")

n_synthetic_1 = int(len(accepted_high_markups) * config.augment_sampling_percentage)
n_synthetic_2 = int(len(rejected_low_markups) * config.augment_sampling_percentage)

print(f"config.augment_sampling_percentage: {config.augment_sampling_percentage}")

print(f"Synthetic samples to generate: {n_synthetic_1} (Accepted High -> Low Markup)")
print(f"Synthetic samples to generate: {n_synthetic_2} (Rejected Low -> High Markup)")

Accepted high markups: 1260
Rejected low markups: 5475
config.augment_sampling_percentage: 0.0
Synthetic samples to generate: 0 (Accepted High -> Low Markup)
Synthetic samples to generate: 0 (Rejected Low -> High Markup)


In [13]:
cleaned_data["type"].value_counts()

type
Quote    49674
Order    11410
Name: count, dtype: int64

In [14]:
cleaned_data["qto"].value_counts()

qto
No     46796
Yes    14288
Name: count, dtype: int64

In [19]:
cleaned_data["request_purpose"].value_counts()

request_purpose
For Bid        42766
For Install    18318
Name: count, dtype: int64

In [20]:
cleaned_data["target"].value_counts()

target
0    49674
1    11410
Name: count, dtype: int64

In [21]:
balanced_train_data["target"].value_counts()

target
0    14936
1     7468
Name: count, dtype: int64

In [18]:
cleaned_data[cleaned_data["target"] == 0]["status"].value_counts()

status
Quote Returned    49674
Name: count, dtype: int64

In [22]:
cleaned_data[cleaned_data["target"] == 0]["lost_reason"].value_counts()

lost_reason
Good Fit Buyer - Not Competitive Pricing                                   163
Used their existing suppliers; Good Fit Buyer - Not Competitive Pricing     57
Used their existing suppliers                                               48
Good Fit Buyer - Not Competitive Pricing; Used their existing suppliers      8
Name: count, dtype: int64

In [23]:
cleaned_data["order_number"].unique()

array(['Q0059-12110', 'Q0059-13808', 'Q0059-14313', ..., 'R0199-21809',
       'R0189-16015-BL', 'R0199-34301'], shape=(4692,), dtype=object)

In [24]:
# Remove Q/R prefixes from order_number
cleaned_data['order_number_clean'] = cleaned_data['order_number'].str.replace('^[QR]', '', regex=True)

# Get unique order numbers for each target group
target_0_orders = set(cleaned_data[cleaned_data["target"] == 0]['order_number_clean'].unique())
target_1_orders = set(cleaned_data[cleaned_data["target"] == 1]['order_number_clean'].unique())

print(f"Unique order numbers in target=0 group: {len(target_0_orders)}")
print(f"Unique order numbers in target=1 group: {len(target_1_orders)}")

# Find overlapping order numbers
overlapping_orders = target_0_orders.intersection(target_1_orders)

print(f"\nNumber of overlapping order numbers: {len(overlapping_orders)}")
print(f"Percentage of target=0 orders that overlap: {len(overlapping_orders)/len(target_0_orders)*100:.2f}%")
print(f"Percentage of target=1 orders that overlap: {len(overlapping_orders)/len(target_1_orders)*100:.2f}%")

if len(overlapping_orders) > 0:
    print(f"\nList of overlapping order numbers (first 20):")
    overlapping_list = sorted(list(overlapping_orders))
    for i, order in enumerate(overlapping_list[:20]):
        print(f"{i+1}. {order}")
    
    if len(overlapping_orders) > 20:
        print(f"... and {len(overlapping_orders) - 20} more")
    
    # Show some examples of the overlapping orders with their targets
    print(f"\nExample records from overlapping orders:")
    sample_order = overlapping_list[0]
    example_records = cleaned_data[cleaned_data['order_number_clean'] == sample_order][['order_number', 'order_number_clean', 'target', 'type', 'status']]
    print(example_records)
else:
    print("\nNo overlapping order numbers found.")

# Additional analysis: show distribution of original Q/R prefixes
print(f"\nOriginal order number prefix distribution:")
cleaned_data['prefix'] = cleaned_data['order_number'].str.extract('^([QR])')
prefix_counts = cleaned_data['prefix'].value_counts()
print(prefix_counts)

print(f"\nPrefix distribution by target:")
prefix_target_crosstab = pd.crosstab(cleaned_data['prefix'], cleaned_data['target'])
print(prefix_target_crosstab)

Unique order numbers in target=0 group: 2762
Unique order numbers in target=1 group: 1930

Number of overlapping order numbers: 0
Percentage of target=0 orders that overlap: 0.00%
Percentage of target=1 orders that overlap: 0.00%

No overlapping order numbers found.

Original order number prefix distribution:
prefix
Q    49674
R    11410
Name: count, dtype: int64

Prefix distribution by target:
target      0      1
prefix              
Q       49674      0
R           0  11410


In [17]:
prepared_quote_order_data

item_id   type request_purpose  qto     order_number  category_id  \
0          1167  Quote     For Install   No      Q0009-16917         11.0   
1          1168  Quote     For Install   No      Q0009-16917         10.0   
2          1169  Quote     For Install   No      Q0009-16917         11.0   
3          1170  Quote     For Install   No      Q0009-16917         11.0   
4          1171  Quote     For Install   No      Q0009-16917         11.0   
...         ...    ...             ...  ...              ...          ...   
323346   104149  Order         For Bid   No      R0149-59806          8.0   
323347   104150  Order         For Bid   No      R0149-59806         11.0   
323348   104151  Order     For Install  Yes  R0149-62413-P12         11.0   
323349   104153  Order     For Install  Yes  R0149-62413-P12          6.0   
323350   104154  Order     For Install  Yes  R0149-62413-P12          7.0   

              plant_category  product_id                       latin_name  \
0                      Trees       216.0  Acer X Freemanii 'Autumn Blaze'   
1                     Shrubs       373.0   Amelanchier Canadensis (Shrub)   
2                      Trees       851.0              Betula Nigra (Tree)   
3                      Trees      1122.0                 Catalpa Speciosa   
4                      Trees      1122.0                 Catalpa Speciosa   
...                      ...         ...                              ...   
323346  Broadleaf Evergreens     12093.0         Kalmia Latifolia 'Sarah'   
323347                 Trees     12525.0            Crataegus Phaenopyrum   
323348                 Trees      8072.0            Rhaphiolepis 'Montic'   
323349    Ornamental Grasses      7131.0                  Spartina Bakeri   
323350            Perennials     24099.0               Strelitzia Reginae   

                                           common_name  ...  order_date_year  \
0           'Jeffersred', 'Autumn Blaze' Freeman Maple  ...             2021   
1                        Canadian Serviceberry (Shrub)  ...             2021   
2                                   River Birch (Tree)  ...             2021   
3                                     Northern Catalpa  ...             2021   
4                                     Northern Catalpa  ...             2021   
...                                                ...  ...              ...   
323346                         'Sarah' Mountain Laurel  ...             2025   
323347                             Washington Hawthorn  ...             2025   
323348               'Majestic Beauty' Indian Hawthorn  ...             2025   
323349                                  Sand Cordgrass  ...             2025   
323350  Crane Flower, Orange Bird of Paradise, Isigude  ...             2025   

       approx_job_date delivery_date    timeline  timeline_month  \
0           2022-07-01           NaN  2022-07-01             Jul   
1           2022-07-01           NaN  2022-07-01             Jul   
2           2022-07-01           NaN  2022-07-01             Jul   
3           2022-07-01           NaN  2022-07-01             Jul   
4           2022-07-01           NaN  2022-07-01             Jul   
...                ...           ...         ...             ...   
323346      2025-09-11           NaN  2025-09-11             Sep   
323347      2025-09-11           NaN  2025-09-11             Sep   
323348      2025-07-11    2025-06-26  2025-06-26             Jun   
323349      2025-07-11    2025-06-26  2025-06-26             Jun   
323350      2025-07-11    2025-06-26  2025-06-26             Jun   

        timeline_year  delivery_method  latitude longitude  lost_reason  
0                2022           Pickup       NaN       NaN          NaN  
1                2022           Pickup       NaN       NaN          NaN  
2                2022           Pickup       NaN       NaN          NaN  
3                2022           Pickup       NaN       NaN          NaN  
4         

In [25]:
target_0_orders

{'0119-37109',
 '0119-56816',
 '0139-53103',
 '0169-13018',
 '0139-94417',
 '0149-58007',
 '0069-05109',
 '0059-22418',
 '0159-02616',
 '0059-62905',
 '0139-06714',
 '0159-49510',
 '0169-70605',
 '0169-91202',
 '0159-83306',
 '0139-12414',
 '0179-41511',
 '0129-34918',
 '0109-71114',
 '0079-35300',
 '0059-63915',
 '0199-00000',
 '0159-59414',
 '0159-08903',
 '0189-20907',
 '0149-70316',
 '0149-00805',
 '0179-29808',
 '0179-31809',
 '0089-14917',
 '0159-82700',
 '0199-01414',
 '0179-30313',
 '0119-34502',
 '0139-10110',
 '0099-68211',
 '0129-55111',
 '0179-34012',
 '0199-26013',
 '0059-19912',
 '0169-93506',
 '0129-22609',
 '0149-63404',
 '0089-87805',
 '0159-01101',
 '0139-58904',
 '0059-27108',
 '0059-37517',
 '0189-25414',
 '0149-14105',
 '0189-00815',
 '0189-72409',
 '0109-52215',
 '0159-78515',
 '0169-25106',
 '0179-84018',
 '0169-08612',
 '0189-21614',
 '0129-59315',
 '0189-34813',
 '0129-64106',
 '0159-64104',
 '0189-05101',
 '0179-17600',
 '0149-96613',
 '0059-54213',
 '0129-632

In [26]:
target_1_orders

{'0069-53802',
 '0169-19810',
 '0159-64508',
 '0049-89613-P6',
 '0099-00518-P15',
 '0169-98600',
 '0109-44615-P4',
 '0159-05406',
 '0169-95002',
 '0159-68510',
 '0119-19908',
 '0149-18309',
 '0139-05805',
 '0169-57305',
 '0089-82408-P1',
 '0159-63700',
 '0139-43218-P1',
 '0149-16712',
 '0159-65215',
 '0019-78215-P1',
 '0189-83506-BL',
 '0079-28811-BL',
 '0159-53411-P3',
 '0089-18212',
 '0159-23800-BL',
 '0179-86303',
 '0099-72718-P3',
 '0099-04501',
 '0069-63706-P1',
 '0169-90211',
 '0139-01601-P7',
 '0119-17503-P2',
 '0119-79818-BL',
 '0189-97109-BL',
 '0149-84708',
 '0129-55717',
 '0079-81203',
 '0079-93815',
 '0159-21414',
 '0099-96812',
 '0119-18513-BL',
 '0069-10506',
 '0099-72112',
 '0059-01013',
 '0139-16618',
 '0089-75010-P3',
 '0159-28812',
 '0189-44010',
 '0179-76115',
 '0179-11313-P2',
 '0139-34305',
 '0109-50801-P4',
 '0079-77504',
 '0109-80614-P1',
 '0139-49909-P13',
 '0149-18814',
 '0139-18013-BL',
 '0189-52904',
 '0149-10406',
 '0119-77918-P3',
 '0079-90015',
 '0139-3291

In [27]:
overlapping_orders

set()

In [29]:
# Simpler and correct approach using regex to extract core order numbers
cleaned_data['order_number_core'] = cleaned_data['order_number'].str.replace('^[QR]', '', regex=True).str.extract(r'(\d+-\d+)')[0]

# Get unique core order numbers for each target group (removing NaN values)
target_0_core = cleaned_data[cleaned_data["target"] == 0]['order_number_core'].dropna().unique()
target_1_core = cleaned_data[cleaned_data["target"] == 1]['order_number_core'].dropna().unique()

target_0_orders = set(target_0_core)
target_1_orders = set(target_1_core)

print(f"Unique core order numbers in target=0 group: {len(target_0_orders)}")
print(f"Unique core order numbers in target=1 group: {len(target_1_orders)}")

# Find overlapping order numbers
overlapping_orders = target_0_orders.intersection(target_1_orders)

print(f"\nNumber of overlapping core order numbers: {len(overlapping_orders)}")
if len(target_0_orders) > 0:
    print(f"Percentage of target=0 orders that overlap: {len(overlapping_orders)/len(target_0_orders)*100:.2f}%")
if len(target_1_orders) > 0:
    print(f"Percentage of target=1 orders that overlap: {len(overlapping_orders)/len(target_1_orders)*100:.2f}%")

if len(overlapping_orders) > 0:
    print(f"\nList of overlapping core order numbers (first 20):")
    overlapping_list = sorted(list(overlapping_orders))
    for i, order in enumerate(overlapping_list[:20]):
        print(f"{i+1}. {order}")
    
    if len(overlapping_orders) > 20:
        print(f"... and {len(overlapping_orders) - 20} more")
    
    # Show examples of overlapping orders
    print(f"\nExample records from overlapping orders:")
    sample_order = overlapping_list[0]
    example_records = cleaned_data[cleaned_data['order_number_core'] == sample_order][['order_number', 'order_number_core', 'target', 'type', 'status']]
    print(example_records)
else:
    print("\nNo overlapping order numbers found.")

# Show some examples of the transformation
print(f"\nExample transformations:")
sample_orders = cleaned_data[['order_number', 'order_number_core']].dropna().drop_duplicates().head(10)
print(sample_orders)

Unique core order numbers in target=0 group: 2762
Unique core order numbers in target=1 group: 1323

Number of overlapping core order numbers: 0
Percentage of target=0 orders that overlap: 0.00%
Percentage of target=1 orders that overlap: 0.00%

No overlapping order numbers found.

Example transformations:
      order_number order_number_core
67032  Q0059-12110        0059-12110
67107  Q0059-13808        0059-13808
67188  Q0059-14313        0059-14313
67292  Q0059-13404        0059-13404
67340  Q0059-16314        0059-16314
67359  Q0059-16901        0059-16901
67384  Q0059-17608        0059-17608
67427  Q0059-17406        0059-17406
67525  Q0059-19912        0059-19912
67526  Q0059-19508        0059-19508


In [30]:
target_1_orders

{'0069-53802',
 '0169-19810',
 '0159-64508',
 '0139-23713',
 '0109-82110',
 '0169-98600',
 '0159-05406',
 '0169-95002',
 '0159-68510',
 '0119-19908',
 '0149-18309',
 '0139-05805',
 '0159-63012',
 '0169-57305',
 '0159-63700',
 '0129-85511',
 '0149-16712',
 '0159-65215',
 '0069-64817',
 '0089-18212',
 '0179-86303',
 '0099-04501',
 '0069-82605',
 '0169-90211',
 '0149-84708',
 '0179-08100',
 '0129-55717',
 '0079-81203',
 '0079-93815',
 '0069-10809',
 '0159-21414',
 '0099-96812',
 '0179-62714',
 '0069-10506',
 '0099-72112',
 '0059-01013',
 '0139-16618',
 '0119-71208',
 '0119-18513',
 '0159-28812',
 '0119-46003',
 '0169-32504',
 '0179-76115',
 '0189-44010',
 '0169-50715',
 '0089-25610',
 '0139-34305',
 '0079-77504',
 '0089-38506',
 '0149-18814',
 '0189-52904',
 '0149-10406',
 '0129-85814',
 '0149-31811',
 '0079-90015',
 '0059-06915',
 '0199-08812',
 '0139-49808',
 '0139-32405',
 '0139-69212',
 '0179-29606',
 '0109-37217',
 '0179-64210',
 '0109-99109',
 '0149-46607',
 '0139-79804',
 '0169-880

In [31]:
cleaned_data

item_id   type request_purpose  qto    order_number  category_id  \
67032    103534  Quote         For Bid   No     Q0059-12110          7.0   
67033    103535  Quote         For Bid   No     Q0059-12110         24.0   
67034    103536  Quote         For Bid   No     Q0059-12110         11.0   
67035    103537  Quote         For Bid   No     Q0059-12110         10.0   
67036    103538  Quote         For Bid   No     Q0059-12110         11.0   
...         ...    ...             ...  ...             ...          ...   
323167   103762  Order     For Install  Yes  R0189-16015-BL          3.0   
323168   103763  Order     For Install  Yes  R0189-16015-BL          8.0   
323171   103770  Order     For Install  Yes  R0189-16015-BL         10.0   
323279   104061  Order     For Install  Yes     R0199-34301          7.0   
323282   104064  Order     For Install  Yes     R0199-34301          6.0   

              plant_category  product_id                           latin_name  \
67032             Perennials      7959.0              Lantana Camara 'Purple'   
67033                  Palms      8308.0                       Sabal Palmetto   
67034                  Trees      7321.0                   Quercus Virginiana   
67035                 Shrubs      8073.0                  Rhaphiolepis Indica   
67036                  Trees      7383.0                 Magnolia Grandiflora   
...                      ...         ...                                  ...   
323167              Conifers     13708.0        Juniperus Chinensis 'Kaizuka'   
323168  Broadleaf Evergreens      7266.0                          Ilex Glabra   
323171                Shrubs     10048.0                Ligustrum Ovalifolium   
323279            Perennials     28095.0              Liriope Muscari 'Aztec'   
323282    Ornamental Grasses      4432.0  Miscanthus Sinensis 'Little Kitten'   

                                              common_name  ...  lost_reason  \
67032                             Lantana Camara 'Purple'  ...          NaN   
67033          Cabbage Palm,  Common Palmetto, Sabal Palm  ...          NaN   
67034                         Live Oak, Southern Oak Tree  ...          NaN   
67035                                     Indian Hawthorn  ...          NaN   
67036                                   Southern Magnolia  ...          NaN   
...                                                   ...  ...          ...   
323167  'Torulosa', 'Hollywood', 'Kaizuka' Chinese Jun...  ...          NaN   
323168                          Inkberry Holly, Gallberry  ...          NaN   
323171                                  California Privet  ...          NaN   
323279                  'Aztec Grass' Variegated Lilyturf  ...          NaN   
323282                       'Little Kitten' Silver-Grass  ...          NaN   

       target accepted_proportion total_seller_price  markup_rate  \
67032       0                0.04             427.50       0.5000   
67033       0                0.04             510.00       0.3529   
67034       0                0.04            1560.00       0.1795   
67035       0                0.04             973.25       0.1765   
67036       0                0.04            3835.00       0.5424   
...       ...                 ...                ...          ...   
323167      1                0.26            4880.00       0.3115   
323168      1                0.26            2160.00       0.1250   
323171      1                0.27            8645.00       0.1538   
323279      1                0.22             591.25       0.2727   
323282      1                0.22             123.75       0.2000   

        markup_amount  total_markup_amount  order_number_clean prefix  \
67032            0.75               213.75          0059-12110      Q   
67033           30.00               180.00          0059-12110      Q   
67034           35.00               280.00          0059-12110      Q   
67035            0.75               171.75          

In [32]:
# Group by order_number_core and count distinct types
order_type_counts = cleaned_data.groupby('order_number_core')['type'].nunique().reset_index()
order_type_counts.columns = ['order_number_core', 'distinct_type_count']

# Display the result
print(f"Shape of new DataFrame: {order_type_counts.shape}")
print("\nFirst 10 rows:")
print(order_type_counts.head(10))

# Show some statistics
print(f"\nStatistics:")
print(f"Total unique order cores: {len(order_type_counts)}")
print(f"Orders with multiple types: {len(order_type_counts[order_type_counts['distinct_type_count'] > 1])}")
print(f"Max distinct types per order: {order_type_counts['distinct_type_count'].max()}")

# Show distribution of distinct type counts
print(f"\nDistribution of distinct type counts:")
print(order_type_counts['distinct_type_count'].value_counts().sort_index())

# Show examples of orders with multiple types
if len(order_type_counts[order_type_counts['distinct_type_count'] > 1]) > 0:
    print(f"\nExamples of orders with multiple types:")
    multi_type_orders = order_type_counts[order_type_counts['distinct_type_count'] > 1].head(5)
    for _, row in multi_type_orders.iterrows():
        order_core = row['order_number_core']
        print(f"\nOrder {order_core}:")
        types_in_order = cleaned_data[cleaned_data['order_number_core'] == order_core]['type'].unique()
        print(f"  Types: {list(types_in_order)}")

Shape of new DataFrame: (4085, 2)

First 10 rows:
  order_number_core  distinct_type_count
0        0019-78215                    1
1        0029-07807                    1
2        0029-30910                    1
3        0039-08305                    1
4        0039-77715                    1
5        0039-77816                    1
6        0049-07206                    1
7        0049-11309                    1
8        0049-11410                    1
9        0049-26509                    1

Statistics:
Total unique order cores: 4085
Orders with multiple types: 0
Max distinct types per order: 1

Distribution of distinct type counts:
distinct_type_count
1    4085
Name: count, dtype: int64


In [ ]:
len(order_type_counts[ > 1])

4085

In [35]:
order_type_counts['distinct_type_count'].max()

np.int64(1)

In [39]:
prepared_quote_order_data.columns

Index(['item_id', 'type', 'request_purpose', 'qto', 'order_number',
       'category_id', 'plant_category', 'product_id', 'latin_name',
       'common_name', 'product_size_id', 'size', 'product_spec_id', 'specs',
       'quantity', 'seller_price', 'buyer_price', 'seller_company_id',
       'supplier_name', 'buyer_company_id', 'buyer_name', 'delivery_address',
       'delivery_address_region', 'delivery_zip_code', 'buyer_region',
       'supplier_address', 'supplier_zip_code', 'status', 'last_publish_date',
       'last_publish_month', 'last_publish_year', 'order_date',
       'order_date_month', 'order_date_year', 'approx_job_date',
       'delivery_date', 'timeline', 'timeline_month', 'timeline_year',
       'delivery_method', 'latitude', 'longitude', 'lost_reason'],
      dtype='object')

In [40]:
# Apply the same logic to prepared_quote_order_data
# Extract core order numbers (remove Q/R prefix and suffixes)
prepared_quote_order_data['order_number_core'] = prepared_quote_order_data['order_number'].str.replace('^[QR]', '', regex=True).str.extract(r'(\d+-\d+)')[0]

# Group by order_number_core and count distinct types
prepared_order_type_counts = prepared_quote_order_data.groupby('order_number_core')['type'].nunique().reset_index()
prepared_order_type_counts.columns = ['order_number_core', 'distinct_type_count']

# Display the result
print(f"Shape of new DataFrame: {prepared_order_type_counts.shape}")
print("\nFirst 10 rows:")
print(prepared_order_type_counts.head(10))

# Show some statistics
print(f"\nStatistics:")
print(f"Total unique order cores: {len(prepared_order_type_counts)}")
print(f"Orders with multiple types: {len(prepared_order_type_counts[prepared_order_type_counts['distinct_type_count'] > 1])}")
print(f"Max distinct types per order: {prepared_order_type_counts['distinct_type_count'].max()}")

# Show distribution of distinct type counts
print(f"\nDistribution of distinct type counts:")
print(prepared_order_type_counts['distinct_type_count'].value_counts().sort_index())

# Show examples of orders with multiple types
if len(prepared_order_type_counts[prepared_order_type_counts['distinct_type_count'] > 1]) > 0:
    print(f"\nExamples of orders with multiple types:")
    multi_type_orders = prepared_order_type_counts[prepared_order_type_counts['distinct_type_count'] > 1].head(5)
    for _, row in multi_type_orders.iterrows():
        order_core = row['order_number_core']
        print(f"\nOrder {order_core}:")
        types_in_order = prepared_quote_order_data[prepared_quote_order_data['order_number_core'] == order_core]['type'].unique()
        print(f"  Types: {list(types_in_order)}")

# Show some examples of the transformation
print(f"\nExample transformations:")
sample_orders = prepared_quote_order_data[['order_number', 'order_number_core']].dropna().drop_duplicates().head(10)
print(sample_orders)

Shape of new DataFrame: (17778, 2)

First 10 rows:
  order_number_core  distinct_type_count
0        0009-10409                    1
1        0009-10510                    1
2        0009-10712                    1
3        0009-10813                    1
4        0009-10914                    1
5        0009-11015                    1
6        0009-11116                    1
7        0009-11217                    1
8        0009-11318                    1
9        0009-11400                    1

Statistics:
Total unique order cores: 17778
Orders with multiple types: 1
Max distinct types per order: 2

Distribution of distinct type counts:
distinct_type_count
1    17777
2        1
Name: count, dtype: int64

Examples of orders with multiple types:

Order 0129-50907:
  Types: ['Quote', 'Order']

Example transformations:
    order_number order_number_core
0    Q0009-16917        0009-16917
73   Q0009-15503        0009-15503
182  Q0009-16008        0009-16008
183  Q0009-17201        0009-1

In [41]:
len(prepared_order_type_counts[prepared_order_type_counts['distinct_type_count'] > 1])

1

In [42]:
prepared_order_type_counts['distinct_type_count'].max()

np.int64(2)

In [45]:
# Read the raw CSV data directly
raw_data = pd.read_csv('data/Quote_and_Order_Data_20250708.csv')

print(f"Raw data shape: {raw_data.shape}")
print(f"Raw data columns: {raw_data.columns.tolist()}")

# Check if 'order_number' and 'type' columns exist
if 'order_number' in raw_data.columns and 'type' in raw_data.columns:
    # Extract core order numbers from raw data (remove Q/R prefix and suffixes)
    raw_data['order_number_core'] = raw_data['order_number'].str.replace('^[QR]', '', regex=True).str.extract(r'(\d+-\d+)')[0]
    
    # Group by order_number_core and count distinct types
    raw_order_type_counts = raw_data.groupby('order_number_core')['type'].nunique().reset_index()
    raw_order_type_counts.columns = ['order_number_core', 'distinct_type_count']
    
    # Display the result
    print(f"\nShape of new DataFrame: {raw_order_type_counts.shape}")
    print("\nFirst 10 rows:")
    print(raw_order_type_counts.head(10))
    
    # Show some statistics
    print(f"\nStatistics:")
    print(f"Total unique order cores: {len(raw_order_type_counts)}")
    print(f"Orders with multiple types: {len(raw_order_type_counts[raw_order_type_counts['distinct_type_count'] > 1])}")
    print(f"Max distinct types per order: {raw_order_type_counts['distinct_type_count'].max()}")
    
    # Show distribution of distinct type counts
    print(f"\nDistribution of distinct type counts:")
    print(raw_order_type_counts['distinct_type_count'].value_counts().sort_index())
    
    # Show examples of orders with multiple types
    if len(raw_order_type_counts[raw_order_type_counts['distinct_type_count'] > 1]) > 0:
        print(f"\nExamples of orders with multiple types:")
        multi_type_orders = raw_order_type_counts[raw_order_type_counts['distinct_type_count'] > 1].head(5)
        for _, row in multi_type_orders.iterrows():
            order_core = row['order_number_core']
            print(f"\nOrder {order_core}:")
            types_in_order = raw_data[raw_data['order_number_core'] == order_core]['type'].unique()
            print(f"  Types: {list(types_in_order)}")
    
    # Show some examples of the transformation
    print(f"\nExample transformations:")
    sample_orders = raw_data[['order_number', 'order_number_core']].dropna().drop_duplicates().head(10)
    print(sample_orders)
    
    # Show type distribution in raw data
    print(f"\nType distribution in raw data:")
    print(raw_data['type'].value_counts())
    
else:
    print("Required columns 'order_number' and/or 'type' not found in raw data")
    print(f"Available columns: {raw_data.columns.tolist()}")

C:\Users\sayed\AppData\Local\Temp\ipykernel_67364\1906794856.py:2: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('data/Quote_and_Order_Data_20250708.csv')


Raw data shape: (323351, 40)
Raw data columns: ['item_id', 'Type', 'Request_Purpose', 'qto', 'Quote_Order_Number', 'category_id', 'Plant_Category', 'product_id', 'Latin_Name', 'Common_Name', 'product_size_id', 'Size', 'product_spec_id', 'Specs', 'QTY', 'Seller_Price', 'Buyer_Price', 'seller_company_id', 'Supplier_Name', 'buyer_company_id', 'Buyer_Name', 'Delivery_Address', 'Delivery_Address_Region', 'Delivery_Address_Zip_Code', 'Buyer_Company_Region', 'Supplier_Address', 'Supplier_Zip_Code', 'Quote_Order_Status', 'Quote_Order_Last_Publish_Date', 'Quote_Order_Last_Publish_Month', 'Quote_Order_Last_Publish_Year', 'Quote_Order_Created_Date', 'Quote_Order_Created_Month', 'Quote_Order_Created_Year', 'approx_job_date', 'Delivery_Date_orders', 'Timeline', 'Timeline_Month', 'Timeline_Year', 'Delivery_method']
Required columns 'order_number' and/or 'type' not found in raw data
Available columns: ['item_id', 'Type', 'Request_Purpose', 'qto', 'Quote_Order_Number', 'category_id', 'Plant_Category',

In [46]:
len(raw_order_type_counts[raw_order_type_counts['distinct_type_count'] > 1])

NameError: name 'raw_order_type_counts' is not defined

In [47]:
raw_data.columns

Index(['item_id', 'Type', 'Request_Purpose', 'qto', 'Quote_Order_Number',
       'category_id', 'Plant_Category', 'product_id', 'Latin_Name',
       'Common_Name', 'product_size_id', 'Size', 'product_spec_id', 'Specs',
       'QTY', 'Seller_Price', 'Buyer_Price', 'seller_company_id',
       'Supplier_Name', 'buyer_company_id', 'Buyer_Name', 'Delivery_Address',
       'Delivery_Address_Region', 'Delivery_Address_Zip_Code',
       'Buyer_Company_Region', 'Supplier_Address', 'Supplier_Zip_Code',
       'Quote_Order_Status', 'Quote_Order_Last_Publish_Date',
       'Quote_Order_Last_Publish_Month', 'Quote_Order_Last_Publish_Year',
       'Quote_Order_Created_Date', 'Quote_Order_Created_Month',
       'Quote_Order_Created_Year', 'approx_job_date', 'Delivery_Date_orders',
       'Timeline', 'Timeline_Month', 'Timeline_Year', 'Delivery_method'],
      dtype='object')

In [48]:
# Read the raw CSV data directly
raw_data = pd.read_csv('data/Quote_and_Order_Data_20250708.csv')

print(f"Raw data shape: {raw_data.shape}")
print(f"Raw data columns: {raw_data.columns.tolist()}")

# Check if 'Quote_Order_Number' and 'Type' columns exist (note the different column names in raw data)
if 'Quote_Order_Number' in raw_data.columns and 'Type' in raw_data.columns:
    # Extract core order numbers from raw data (remove Q/R prefix and suffixes)
    raw_data['order_number_core'] = raw_data['Quote_Order_Number'].str.replace('^[QR]', '', regex=True).str.extract(r'(\d+-\d+)')[0]
    
    # Group by order_number_core and count distinct types
    raw_order_type_counts = raw_data.groupby('order_number_core')['Type'].nunique().reset_index()
    raw_order_type_counts.columns = ['order_number_core', 'distinct_type_count']
    
    # Display the result
    print(f"\nShape of new DataFrame: {raw_order_type_counts.shape}")
    print("\nFirst 10 rows:")
    print(raw_order_type_counts.head(10))
    
    # Show some statistics
    print(f"\nStatistics:")
    print(f"Total unique order cores: {len(raw_order_type_counts)}")
    print(f"Orders with multiple types: {len(raw_order_type_counts[raw_order_type_counts['distinct_type_count'] > 1])}")
    print(f"Max distinct types per order: {raw_order_type_counts['distinct_type_count'].max()}")
    
    # Show distribution of distinct type counts
    print(f"\nDistribution of distinct type counts:")
    print(raw_order_type_counts['distinct_type_count'].value_counts().sort_index())
    
    # Show examples of orders with multiple types
    if len(raw_order_type_counts[raw_order_type_counts['distinct_type_count'] > 1]) > 0:
        print(f"\nExamples of orders with multiple types:")
        multi_type_orders = raw_order_type_counts[raw_order_type_counts['distinct_type_count'] > 1].head(5)
        for _, row in multi_type_orders.iterrows():
            order_core = row['order_number_core']
            print(f"\nOrder {order_core}:")
            types_in_order = raw_data[raw_data['order_number_core'] == order_core]['Type'].unique()
            print(f"  Types: {list(types_in_order)}")
    
    # Show some examples of the transformation
    print(f"\nExample transformations:")
    sample_orders = raw_data[['Quote_Order_Number', 'order_number_core']].dropna().drop_duplicates().head(10)
    print(sample_orders)
    
    # Show type distribution in raw data
    print(f"\nType distribution in raw data:")
    print(raw_data['Type'].value_counts())
    
else:
    print("Required columns 'Quote_Order_Number' and/or 'Type' not found in raw data")
    print(f"Available columns: {raw_data.columns.tolist()}")

C:\Users\sayed\AppData\Local\Temp\ipykernel_67364\409303789.py:2: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('data/Quote_and_Order_Data_20250708.csv')


Raw data shape: (323351, 40)
Raw data columns: ['item_id', 'Type', 'Request_Purpose', 'qto', 'Quote_Order_Number', 'category_id', 'Plant_Category', 'product_id', 'Latin_Name', 'Common_Name', 'product_size_id', 'Size', 'product_spec_id', 'Specs', 'QTY', 'Seller_Price', 'Buyer_Price', 'seller_company_id', 'Supplier_Name', 'buyer_company_id', 'Buyer_Name', 'Delivery_Address', 'Delivery_Address_Region', 'Delivery_Address_Zip_Code', 'Buyer_Company_Region', 'Supplier_Address', 'Supplier_Zip_Code', 'Quote_Order_Status', 'Quote_Order_Last_Publish_Date', 'Quote_Order_Last_Publish_Month', 'Quote_Order_Last_Publish_Year', 'Quote_Order_Created_Date', 'Quote_Order_Created_Month', 'Quote_Order_Created_Year', 'approx_job_date', 'Delivery_Date_orders', 'Timeline', 'Timeline_Month', 'Timeline_Year', 'Delivery_method']

Shape of new DataFrame: (17778, 2)

First 10 rows:
  order_number_core  distinct_type_count
0        0009-10409                    1
1        0009-10510                    1
2        000

In [49]:
len(raw_order_type_counts[raw_order_type_counts['distinct_type_count'] > 1])

1

In [51]:
raw_order_type_counts['distinct_type_count'].max()

np.int64(2)

In [52]:
raw_order_type_counts[raw_order_type_counts['distinct_type_count'] > 1]

order_number_core  distinct_type_count
11242        0129-50907                    2